In [7]:
import scipy.stats as stats
import numpy as np
import pymc3 as pm

In [8]:
#Actual probabilities
p_A = 0.05
p_B = 0.04

#User Traffic
n_users = 13500
n_A = stats.binom.rvs(n=n_users, p=0.5, size=1)[0]
n_B = n_users - n_A

#Conversions
conversions_A = stats.bernoulli.rvs(p_A, size=n_A)
conversions_B = stats.bernoulli.rvs(p_B, size=n_B)

print("creative A was observed {} times and led to {} conversions".format(n_A, sum(conversions_A)))
print("creative B was observed {} times and led to {} conversions".format(n_B, sum(conversions_B)))

creative A was observed 6731 times and led to 315 conversions
creative B was observed 6769 times and led to 290 conversions


In [9]:
with pm.Model() as model:
    n_users = 10000
    
    #Define Stochastic and Deterministic Variable (Build the network)
    #Amount of users
    n_A = pm.Binomial("n_A", n_users, 0.5)
    n_B = pm.Deterministic("n_B", n_users - n_A)
    
    # Amount of conversions
    conversions_A = pm.Binomial("conversions_A", n_A, p_A)
    conversions_B = pm.Binomial("conversions_B", n_B, p_B)
    
    observed_conversions_A = pm.Deterministic('observed_conversions_A', conversions_A)
    observed_conversions_B = pm.Deterministic('observed_conversions_B', conversions_B)
    
    p_estimates = pm.Uniform("p_estimates", 0, 1, shape=2)
    delta = pm.Deterministic("delta", p_estimates[1] - p_estimates[0])

    #Feed the network with observations
    obs_A = pm.Binomial("obs_A", n_A, p_estimates[0], observed=observed_conversions_A)
    obs_B = pm.Binomial("obs_B", n_B, p_estimates[1], observed=observed_conversions_B)

    #Run MCMC algorithm
    start = pm.find_MAP()
    step = pm.Metropolis()
    trace = pm.sample(50000, step=step)
    burned_trace = trace[1000:]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [p_estimates]
>Metropolis: [conversions_B]
>Metropolis: [conversions_A]
>Metropolis: [n_A]


/opt/conda/lib/python3.7/site-packages/pymc3/step_methods/metropolis.py:226: RuntimeWarning: overflow encountered in exp
  "accept": np.exp(accept),
/opt/conda/lib/python3.7/site-packages/pymc3/step_methods/metropolis.py:226: RuntimeWarning: overflow encountered in exp
  "accept": np.exp(accept),
/opt/conda/lib/python3.7/site-packages/pymc3/step_methods/metropolis.py:226: RuntimeWarning: overflow encountered in exp
  "accept": np.exp(accept),
/opt/conda/lib/python3.7/site-packages/pymc3/step_methods/metropolis.py:226: RuntimeWarning: overflow encountered in exp
  "accept": np.exp(accept),
Sampling 4 chains for 1_000 tune and 50_000 draw iterations (4_000 + 200_000 draws total) took 34 seconds.
The number of effective samples is smaller than 10% for some parameters.


In [10]:
np.mean(delta_samples>0)

NameError: name 'delta_samples' is not defined

In [14]:
tau = pm.Deterministic("tau", p_estimates[0] / p_estimates[1])

TypeError: No model on context stack.